In [1]:
from PIL import Image
from pathlib import Path
import numpy as np
import os

os.chdir(r"C:\Users\josef\OneDrive\Documentos\metodos\TP2_Metodos")
print("Ahora estás en:", os.getcwd())


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\josef\\OneDrive\\Documentos\\metodos\\TP2_Metodos'

Función para vectorizar la imagen en 256x256x1

In [22]:
def vectorizar_imagen(ruta, size=(256, 256), normalizar=False):
    img = Image.open(ruta) 
    img = img.resize(size) # redimensiona la imagen
    gray = img.convert('L') # transforma a escala de grises
    vector = np.array(gray).flatten().reshape(-1, 1)
    
    if normalizar: # normalizar el vector
        vector = vector / 255.0

    return vector

Función para calcular la predicción del modelo

In [23]:
def prediccion(i, w, b):
    z = np.dot(w.T, i) + b  # w.T tiene shape (1, 65536)
    return (np.tanh(z) + 1) / 2

Función para calcular el error cuadrático dado un w y b, utilizando el modelo propuesto

In [24]:
def error_cuadratico(carpeta, w, b, size=(256, 256), normalizar=False):
    error_total = 0.0
    contador = 0
    
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            # si tiene parkison, el diagnóstico es positivo (1), sino es negativo (0)
            if archivo.startswith("Healthy"): 
                d = 0
            elif archivo.startswith("Parkinson"):
                d = 1
            ruta = os.path.join(carpeta, archivo)
            i = vectorizar_imagen(ruta, size=size, normalizar=normalizar)
            pred = prediccion(i, w, b) # calculo la predicción
            error_total += (pred - d) ** 2 # sumo el error parcial al total
            contador += 1

    return error_total / contador

Función para calcular la derivada parcial contra w de la función de pérdida

In [25]:
def derivada_parcial_w(carpeta, w, b, size=(256, 256), normalizar=False):
    derivada_w_total = 0.0
    contador = 0
    
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            if archivo.startswith("Healthy"):
                d = 0
            elif archivo.startswith("Parkinson"):
                d = 1
            ruta = os.path.join(carpeta, archivo)
            i = vectorizar_imagen(ruta, size=size, normalizar=normalizar)
            ruta = os.path.join(carpeta, archivo)
            z = np.dot(w.T, i) + b
            t0 = np.tanh(z)
            pred = (t0 + 1) / 2
            derivada_w_total += (1 - t0**2) * 0.5 * (pred - d) * i
            contador += 1

    if contador == 0:
        return np.zeros_like(w)
    
    return derivada_w_total / contador

Función para calcular la derivada parcial contra b de la función de pérdida

In [26]:
def derivada_parcial_b(carpeta, w, b, size=(256, 256), normalizar=False):
    derivada_b_total = 0.0
    contador = 0

    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            if archivo.startswith("Healthy"):
                d = 0
            elif archivo.startswith("Parkinson"):
                d = 1
            ruta = os.path.join(carpeta, archivo)
            i = vectorizar_imagen(ruta, size=size, normalizar=normalizar)
            z = np.dot(w.T, i) + b
            t0 = np.tanh(z)
            pred = (t0 + 1) / 2
            derivada_b_total += (1 - t0**2) * 0.5 * (pred - d)
            contador += 1
            
    if contador == 0:
        return 0.0
    
    return derivada_b_total / contador

Función para aplicar la regla de actualización para w y b

In [27]:
def actualizacion(carpeta, w, b, alpha, size=(256, 256), normalizar=False):
    derivada_w = derivada_parcial_w(carpeta, w, b, size=size, normalizar=normalizar)
    derivada_b = derivada_parcial_b(carpeta, w, b, size=size, normalizar=normalizar)
    w_nuevo = w - alpha * derivada_w
    b_nuevo = b - alpha * derivada_b
    return w_nuevo, b_nuevo

Función para recorrer por una única iteración el algoritmo de búsqueda

In [28]:
def entrenar_iteracion_unica(carpeta, w_inicial, b_inicial, alpha, size=(256, 256), normalizar=False):
    w = w_inicial
    b = b_inicial

    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            ruta = os.path.join(carpeta, archivo)
            i = vectorizar_imagen(ruta, size=size, normalizar=normalizar)
            (w, b) = actualizacion(carpeta, w, b, alpha, size=size, normalizar=normalizar)

    return w, b

Función para repetir el método de descenso de gradiente por cierta cantidad de iteraciones

In [29]:
def entrenar_descenso_gradiente(carpeta, w_inicial, b_inicial, alpha, size=(256, 256), normalizar=False, repeticiones=1000, epsilon=1e-6):
    w = w_inicial.copy()
    b = b_inicial
    
    # Listas para almacenar el historial de errores
    errores_entrenamiento = []
    
    error_anterior = error_cuadratico(carpeta, w, b, size=size, normalizar=normalizar)
    errores_entrenamiento.append(error_anterior)
    
    print(f"Error inicial: {error_anterior}")
    
    for i in range(repeticiones):
        # Actualizar parámetros
        w, b = actualizacion(carpeta, w, b, alpha, size=size, normalizar=normalizar)
        
        # Calcular nuevo error
        error_actual = error_cuadratico(carpeta, w, b, size=size, normalizar=normalizar)
        errores_entrenamiento.append(error_actual)
        
        # Mostrar progreso cada 100 iteraciones
        if i % 100 == 0:
            print(f"Iteración {i}, Error: {error_actual}")
        
        # Verificar convergencia
        if abs(error_actual - error_anterior) < epsilon:
            print(f"Convergencia alcanzada en la iteración {i}")
            break
        
        error_anterior = error_actual
    
    return w, b, errores_entrenamiento

In [30]:
# Ruta relativa segura desde donde está el notebook (Jupyter usa cwd)
carpeta_imagenes = os.path.join(".", "Entrenamiento")
print("Estoy parado en:", os.getcwd())


n = 256*256
w = np.random.randn(n, 1) * (1 / np.sqrt(n))
b = 0.0
entrenar_descenso_gradiente(carpeta_imagenes, w_inicial=w, b_inicial=b, alpha=0.01, repeticiones=10, size=(256, 256), normalizar=False)
# NO SE QUE HACER CON ESTA PARTE

Estoy parado en: C:\Users\josef\OneDrive\Documentos\metodos\TP2_Metodos


KeyboardInterrupt: 

In [ ]:
base = os.getcwd()


In [ ]:
def calcular_accuracy(carpeta, w, b, size=(256, 256), normalizar=False, umbral=0.5):
    correctos = 0
    total = 0
    
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            if archivo.startswith("Healthy"): 
                d = 0
            elif archivo.startswith("Parkinson"):
                d = 1            
            ruta = os.path.join(carpeta, archivo)
            i = vectorizar_imagen(ruta, size=size, normalizar=normalizar)
            pred = prediccion(i, w, b)
            pred_clase = 1 if pred > umbral else 0
                
            if pred_clase == d:
                correctos += 1
            total += 1
    
    if total == 0:
        return 0.0
    
    return correctos / total

In [ ]:
def matriz_confusion(carpeta, w, b, size=(256, 256), normalizar=False, umbral=0.5):
    vp = 0
    fp = 0
    vn = 0
    fn = 0
    
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            if archivo.startswith("Healthy"): 
                d = 0
            elif archivo.startswith("Parkinson"):
                d = 1

            ruta = os.path.join(carpeta, archivo)
            i = vectorizar_imagen(ruta, size=size, normalizar=normalizar)
            pred = prediccion(i, w, b)
            pred_clase = 1 if pred > umbral else 0

            if d == 1 and pred_clase == 1:
                vp += 1
            elif d == 0 and pred_clase == 1:
                fp += 1
            elif d == 0 and pred_clase == 0:
                vn += 1
            elif d == 1 and pred_clase == 0:
                fn += 1

    return vp, fp, vn, fn

In [ ]:
def visualizar_entrenamiento(errores):
    plt.figure(figsize=(10, 6))
    plt.plot(errores)
    plt.title('Evolución del Error Durante el Entrenamiento')
    plt.xlabel('Iteración')
    plt.ylabel('Error Cuadrático')
    plt.grid(True)
    plt.show()

In [ ]:
def ejecutar_entrenamiento(carpeta_entrenamiento, carpeta_testing=None, size=(256, 256), normalizar=False, alpha=0.01, repeticiones=1000):
    # Verificar que la carpeta existe
    if not os.path.exists(carpeta_entrenamiento):
        print(f"Error: La carpeta {carpeta_entrenamiento} no existe")
        print(f"Directorio actual: {os.getcwd()}")
        print("Archivos y carpetas disponibles:")
        for item in os.listdir('.'):
            print(f"  - {item}")
        return None, None, None

    # Inicializar parámetros
    n = size[0] * size[1]
    w = np.random.randn(n, 1) * (1 / np.sqrt(n))
    b = 0.0
    
    print(f"Iniciando entrenamiento con:")
    print(f"  - Tamaño de imagen: {size}")
    print(f"  - Normalización: {normalizar}")
    print(f"  - Alpha: {alpha}")
    print(f"  - Repeticiones: {repeticiones}")
    print(f"  - Dimensión de w: {w.shape}")

    # Entrenar el modelo
    w_final, b_final, errores = entrenar_descenso_gradiente(carpeta_entrenamiento, w, b, alpha, size=size, 
                                                            normalizar=normalizar, repeticiones=repeticiones)

    # Calcular accuracy en entrenamiento
    acc_entrenamiento = calcular_accuracy(carpeta_entrenamiento, w_final, b_final, size=size, normalizar=normalizar)
    print(f"Accuracy en entrenamiento: {acc_entrenamiento:.4f}")

    # Calcular accuracy en testing si se proporciona
    if carpeta_testing and os.path.exists(carpeta_testing):
        acc_testing = calcular_accuracy(carpeta_testing, w_final, b_final, size=size, normalizar=normalizar)
        print(f"Accuracy en testing: {acc_testing:.4f}")

    # Visualizar entrenamiento
    visualizar_entrenamiento(errores)
    
    return w_final, b_final, errores


In [31]:
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

# (Incluí acá todas tus funciones: vectorizar_imagen, prediccion, error_cuadratico, derivada_parcial_w, derivada_parcial_b, actualizacion, calcular_accuracy, matriz_confusion)

# --- Copiá acá todas tus funciones anteriores ---

# Para no repetir todo, asumamos que ya las tenés definidas arriba

if __name__ == "__main__":
    carpeta_entrenamiento = r"C:\Users\josef\OneDrive\Documentos\metodos\TP2_Metodos\Entrenamiento"
    
    # Tomar una imagen de ejemplo
    archivos = [f for f in os.listdir(carpeta_entrenamiento) if f.lower().endswith(('.png','.jpg','.jpeg'))]
    if not archivos:
        print("No se encontraron imágenes en la carpeta de entrenamiento")
        exit()
    ruta_ejemplo = os.path.join(carpeta_entrenamiento, archivos[0])
    
    print("Probando vectorizar_imagen...")
    vector = vectorizar_imagen(ruta_ejemplo, size=(64,64), normalizar=True)
    print("Vector shape:", vector.shape)
    print("Valores min y max:", vector.min(), vector.max())
    
    print("\nProbando prediccion...")
    w_test = np.random.randn(64*64, 1)
    b_test = 0.0
    i_test = np.random.rand(64*64, 1)
    pred = prediccion(i_test, w_test, b_test)
    print("Predicción ejemplo:", pred)
    
    print("\nProbando error_cuadratico...")
    error = error_cuadratico(carpeta_entrenamiento, w_test, b_test, size=(64,64), normalizar=True)
    print("Error cuadrático ejemplo:", error)
    
    print("\nProbando derivadas parciales...")
    dw = derivada_parcial_w(carpeta_entrenamiento, w_test, b_test, size=(64,64), normalizar=True)
    db = derivada_parcial_b(carpeta_entrenamiento, w_test, b_test, size=(64,64), normalizar=True)
    print("Derivada parcial w shape:", dw.shape)
    print("Derivada parcial b:", db)
    
    print("\nProbando actualización...")
    w_n, b_n = actualizacion(carpeta_entrenamiento, w_test, b_test, alpha=0.001, size=(64,64), normalizar=True)
    print("w nuevo shape:", w_n.shape)
    print("b nuevo:", b_n)
    
    print("\nProbando accuracy...")
    acc = calcular_accuracy(carpeta_entrenamiento, w_test, b_test, size=(64,64), normalizar=True)
    print("Accuracy ejemplo:", acc)
    
    print("\nProbando matriz de confusión...")
    vp, fp, vn, fn = matriz_confusion(carpeta_entrenamiento, w_test, b_test, size=(64,64), normalizar=True)
    print(f"VP: {vp}, FP: {fp}, VN: {vn}, FN: {fn}")

Probando vectorizar_imagen...
Vector shape: (4096, 1)
Valores min y max: 0.47843137254901963 0.9725490196078431

Probando prediccion...
Predicción ejemplo: [[1.]]

Probando error_cuadratico...
Error cuadrático ejemplo: [[0.5]]

Probando derivadas parciales...
Derivada parcial w shape: (4096, 1)
Derivada parcial b: [[0.]]

Probando actualización...
w nuevo shape: (4096, 1)
b nuevo: [[0.]]

Probando accuracy...
Accuracy ejemplo: 0.5

Probando matriz de confusión...


NameError: name 'tp' is not defined